In [1]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import csv
import time
from csv import DictWriter

DOMAIN="https://fbref.com"
URL = 'https://fbref.com/en/countries/'

response = rq.get(URL)

def get_soup(url):
    return bs(rq.get(url).text,"html.parser")

soup=get_soup(URL)

In [2]:
def findtableworlcupmen(link):
    if 'href' in link.attrs and link.text=='Men': #just Men
        #print(str(DOMAIN+link.attrs['href'])+"\n")
        URLmen=DOMAIN+link.attrs['href']
        #go to this page
        response = rq.get(URLmen)
        #print(URLmen)
        soup=get_soup(URLmen) #page html du pays
        table=soup.find('table',{'id':'comps_cup_1'})
        return table

In [3]:
def findcountryandlinks(tr):
    if tr.find('strong'):
        country=tr.find('strong') #pays a enregistrer dans pandas
        country=country.text
        tds=tr.find('td',{"data-stat":"national_teams"})
        #print(tds)
        links=tds.find_all('a')
        return country,links
    else:
        return None

In [4]:
def findsquad(row):
    thannee=row.find('th') #annee a enregistrer dans pandas
    annee=thannee.text
    #print(thannee.text)
    tdsannee=row.find('td',{'data-stat':"squad"})
    return annee,tdsannee

In [5]:
def savetablecsv(table,country,annee):
    rows = table.find_all("tr")
    for row in rows:
        datematch=None
        tdlinkmatch=None
        dict_infos_match={}
        for cell in row.findAll(["td", "th"]):
            tdlinkmatch=row.find('td',{"data-stat":"match_report"})
            date=row.find('th',{"data-stat":"date"})
            opponent=row.find('td',{"data-stat":"opponent"})
        #print(tdlinkmatch)
        if tdlinkmatch!=None:
            matchreportpage(tdlinkmatch,date,opponent,country) #dict with infos from match we add it to column in file

In [6]:
def matchreporttables(soupmatch,dateexacte,opponentname,country):
    #print("Country: " + country + " Opponent: " + opponentname)
    
    divsmatch=soupmatch.find_all('div',{'class':'table_container current'})
    #tablesmatch=soupmatch.find_all('table')
    if divsmatch!=None:
        #if (tablesmatch.find('table')!=None):
        for div in divsmatch: #enregistrer chaque table sous format csv
            #prendre caption de la table
            #print(tab)
            tab=div.find('table')
            
            if tab.caption!=None:
                caption=tab.caption.text
                
                file= "PlayersStatsMatch/" + caption + "_" + dateexacte + "_" + opponentname + "_PlayersMatch.csv"
                rows = tab.find_all("tr")
                with open(file, "wt+", encoding="utf-8",newline="") as f:
                    writer = csv.writer(f)
                    for row in rows:
                        csv_row = []
                        for cell in row.findAll(["td", "th"]):
                            csv_row.append(cell.get_text())

                        writer.writerow(csv_row)

In [7]:
def matchreportpage(tdlinkmatch,date,opponent,country):
     #take a href 
     #DOMAIN+"en/matches/16253d86/Belarus-France-September-6-2016-WCQ----UEFA-M"
    if 'href' in tdlinkmatch.find('a').attrs:
        URLmatch=DOMAIN+tdlinkmatch.find('a').attrs['href']
        #print(URLmatch)
        #go to this page
        response = rq.get(URLmatch)
        #print(URLmen)
        soupmatch=get_soup(URLmatch) #page html du match
        #get date and egt opponent
        if date.find('a'):
            dateexacte=date.a.text
            #print(dateexacte)
        if opponent.find('a'):
            opponentname=opponent.a.text
            
        matchreporttables(soupmatch,dateexacte,opponentname,country)

In [8]:
def findallstats(linkannee):
    URLannee=DOMAIN+linkannee.attrs['href']
    #print(URLannee)
    response = rq.get(URLannee)
    soupannee=get_soup(URLannee)
    tableannee=soupannee.find('table',{'id':'matchlogs_for'})
    return tableannee

In [9]:
def iteratelines(trs):
    for tr in trs:
        #nom pays
        if findcountryandlinks(tr)!=None:
            country,links=findcountryandlinks(tr)
            #trouver liens joueurs
            if links!=None:
                for link in links:
                    table=findtableworlcupmen(link)
                    if table!=None:
                        for row in table.find_all('tr'):
                            annee,tdsannee=findsquad(row)
                            if tdsannee!=None:
                                linksyear=tdsannee.find_all('a')
                                #print(linksyear)
                                for linkannee in linksyear:
                                    #print(linkannee)
                                    tableannee=findallstats(linkannee)
                                    if tableannee!=None:
                                        if annee>='2000':
                                        
                                            savetablecsv(tableannee,country,annee)


In [10]:
trs=soup.find_all('tr')
iteratelines(trs)